In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Linear Regression Examplt").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/09 21:59:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/09 21:59:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [4]:
data = spark.read.csv("cruise_ship_info.csv", header=True, inferSchema=True)
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [16]:
stringIndexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_indexed")

In [17]:
stringIndexerModel = stringIndexer.fit(data)

In [18]:
new_data = stringIndexerModel.transform(data)
new_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.5

In [36]:
new_data.select("Cruise_line_indexed").distinct().collect()

[Row(Cruise_line_indexed=8.0),
 Row(Cruise_line_indexed=0.0),
 Row(Cruise_line_indexed=7.0),
 Row(Cruise_line_indexed=18.0),
 Row(Cruise_line_indexed=1.0),
 Row(Cruise_line_indexed=4.0),
 Row(Cruise_line_indexed=11.0),
 Row(Cruise_line_indexed=14.0),
 Row(Cruise_line_indexed=3.0),
 Row(Cruise_line_indexed=19.0),
 Row(Cruise_line_indexed=2.0),
 Row(Cruise_line_indexed=17.0),
 Row(Cruise_line_indexed=10.0),
 Row(Cruise_line_indexed=13.0),
 Row(Cruise_line_indexed=6.0),
 Row(Cruise_line_indexed=5.0),
 Row(Cruise_line_indexed=15.0),
 Row(Cruise_line_indexed=9.0),
 Row(Cruise_line_indexed=16.0),
 Row(Cruise_line_indexed=12.0)]

In [40]:
data.select("Cruise_line").distinct().collect()

[Row(Cruise_line='Costa'),
 Row(Cruise_line='P&O'),
 Row(Cruise_line='Cunard'),
 Row(Cruise_line='Regent_Seven_Seas'),
 Row(Cruise_line='MSC'),
 Row(Cruise_line='Carnival'),
 Row(Cruise_line='Crystal'),
 Row(Cruise_line='Orient'),
 Row(Cruise_line='Princess'),
 Row(Cruise_line='Silversea'),
 Row(Cruise_line='Seabourn'),
 Row(Cruise_line='Holland_American'),
 Row(Cruise_line='Windstar'),
 Row(Cruise_line='Disney'),
 Row(Cruise_line='Norwegian'),
 Row(Cruise_line='Oceania'),
 Row(Cruise_line='Azamara'),
 Row(Cruise_line='Celebrity'),
 Row(Cruise_line='Star'),
 Row(Cruise_line='Royal_Caribbean')]

In [19]:
new_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_indexed']

In [20]:
assembler = VectorAssembler(
    inputCols=[
        "Age",
        "Tonnage",
        "passengers",
        "length",
        "cabins",
        "passenger_density",
        "Cruise_line_indexed",
    ],
    outputCol="features",
)

In [21]:
output = assembler.transform(new_data)

In [45]:
output.show(truncate=False)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+--------------------------------------------------+
|Ship_name  |Cruise_line|Age|Tonnage           |passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|features                                          |
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+--------------------------------------------------+
|Journey    |Azamara    |6  |30.276999999999997|6.94      |5.94  |3.55  |42.64            |3.55|16.0               |[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|
|Quest      |Azamara    |6  |30.276999999999997|6.94      |5.94  |3.55  |42.64            |3.55|16.0               |[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|
|Celebration|Carnival   |26 |47.262            |14.86     |7.22  |7.43  |31.8             |6.7 |1.0                |[26.0,47.262,14.86,7.22,7.43,31.8,1.0]      

In [46]:
output.select("features", "crew").show(truncate=False)

+--------------------------------------------------+----+
|features                                          |crew|
+--------------------------------------------------+----+
|[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|3.55|
|[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|3.55|
|[26.0,47.262,14.86,7.22,7.43,31.8,1.0]            |6.7 |
|[11.0,110.0,29.74,9.53,14.88,36.99,1.0]           |19.1|
|[17.0,101.353,26.42,8.92,13.21,38.36,1.0]         |10.0|
|[22.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[15.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[23.0,70.367,20.56,8.55,10.22,34.23,1.0]          |9.2 |
|[19.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[6.0,110.23899999999999,37.0,9.51,14.87,29.79,1.0]|11.5|
|[10.0,110.0,29.74,9.51,14.87,36.99,1.0]           |11.6|
|[28.0,46.052,14.52,7.27,7.26,31.72,1.0]           |6.6 |
|[18.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[17.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[11.0,86.0,21

In [23]:
final_data = output.select("features", "crew")

In [24]:
final_data.show(truncate=False)

+--------------------------------------------------+----+
|features                                          |crew|
+--------------------------------------------------+----+
|[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|3.55|
|[6.0,30.276999999999997,6.94,5.94,3.55,42.64,16.0]|3.55|
|[26.0,47.262,14.86,7.22,7.43,31.8,1.0]            |6.7 |
|[11.0,110.0,29.74,9.53,14.88,36.99,1.0]           |19.1|
|[17.0,101.353,26.42,8.92,13.21,38.36,1.0]         |10.0|
|[22.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[15.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[23.0,70.367,20.56,8.55,10.22,34.23,1.0]          |9.2 |
|[19.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[6.0,110.23899999999999,37.0,9.51,14.87,29.79,1.0]|11.5|
|[10.0,110.0,29.74,9.51,14.87,36.99,1.0]           |11.6|
|[28.0,46.052,14.52,7.27,7.26,31.72,1.0]           |6.6 |
|[18.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[17.0,70.367,20.52,8.55,10.2,34.29,1.0]           |9.2 |
|[11.0,86.0,21

In [48]:
train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=20)

In [49]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              117|
|   mean|7.908717948717958|
| stddev| 3.57895851216946|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [50]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                41|
|   mean| 7.467317073170732|
| stddev|3.2989801639220593|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [51]:
lr = LinearRegression(labelCol="crew")

In [52]:
lr_model = lr.fit(train_data)

24/03/09 22:34:35 WARN Instrumentation: [ad68b11c] regParam is zero, which might cause numerical instability and overfitting.


In [53]:
test_resutl = lr_model.evaluate(test_data)

In [54]:
test_resutl.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.2752564451201476|
| -0.5692223935878893|
|-0.33845256383708566|
| 0.38940107674238256|
| -0.5301038905684692|
|  -0.519562322725184|
| -0.2795865744832291|
|  1.5840737513377299|
|  0.1948040239725053|
|  0.9979918810633261|
| -0.4860850374985084|
|  0.0785318113819109|
| -0.4862305861429297|
|-0.37805325953321933|
| 0.18558457632976788|
|   3.198106482993822|
| -1.0649390905809817|
| 0.10756742361880534|
|  -1.731069877483522|
| 0.14439957887712573|
+--------------------+
only showing top 20 rows



In [55]:
test_resutl.rootMeanSquaredError

0.8060419594818015

In [56]:
test_resutl.r2

0.9388100980684839

In [57]:
test_resutl.r2adj

0.9258304219011926

In [58]:
unlabeled_data = test_data.select("features")

In [59]:
unlabeled_data.show(truncate=False)

+---------------------------------------------------+
|features                                           |
+---------------------------------------------------+
|[5.0,115.0,35.74,9.0,15.32,32.18,8.0]              |
|[6.0,112.0,38.0,9.51,15.0,29.47,5.0]               |
|[7.0,158.0,43.7,11.12,18.0,36.16,0.0]              |
|[8.0,77.499,19.5,8.56,9.75,39.74,2.0]              |
|[9.0,105.0,27.2,8.9,13.56,38.6,5.0]                |
|[9.0,110.0,29.74,9.52,14.87,36.99,1.0]             |
|[10.0,58.825,15.6,8.23,7.65,37.71,7.0]             |
|[10.0,151.4,26.2,11.32,11.34,57.79,12.0]           |
|[11.0,58.6,15.66,8.23,7.83,37.42,7.0]              |
|[11.0,90.0,22.4,9.65,11.2,40.18,4.0]               |
|[11.0,90.09,25.01,9.62,10.5,36.02,0.0]             |
|[13.0,30.276999999999997,6.84,5.94,3.42,44.26,13.0]|
|[13.0,61.0,13.8,7.8,6.88,44.2,3.0]                 |
|[13.0,85.619,21.14,9.57,10.56,40.5,5.0]            |
|[14.0,33.0,4.9,5.6,2.45,67.35,10.0]                |
|[14.0,76.8,19.6,8.79,9.67,3

In [60]:
prediction = lr_model.transform(unlabeled_data)

In [61]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,115.0,35.74,...|11.924743554879852|
|[6.0,112.0,38.0,9...| 11.46922239358789|
|[7.0,158.0,43.7,1...|13.938452563837085|
|[8.0,77.499,19.5,...| 8.610598923257617|
|[9.0,105.0,27.2,8...|11.210103890568469|
|[9.0,110.0,29.74,...|12.119562322725184|
|[10.0,58.825,15.6...| 7.279586574483229|
|[10.0,151.4,26.2,...| 10.94592624866227|
|[11.0,58.6,15.66,...| 7.405195976027494|
|[11.0,90.0,22.4,9...|10.002008118936674|
|[11.0,90.09,25.01...| 8.966085037498509|
|[13.0,30.27699999...| 3.921468188618089|
|[13.0,61.0,13.8,7...|  6.48623058614293|
|[13.0,85.619,21.1...| 9.578053259533219|
|[14.0,33.0,4.9,5....|3.0544154236702323|
|[14.0,76.8,19.6,8...| 8.801893517006178|
|[14.0,138.0,31.14...|12.824939090580981|
|[15.0,30.27699999...|3.8924325763811947|
|[15.0,78.491,24.3...| 8.331069877483522|
|[15.0,108.806,26....|10.955600421122874|
+--------------------+------------

In [62]:
from pyspark.sql.functions import corr

In [64]:
data.select(corr("crew", "passengers")).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [65]:
data.select(corr("crew", "cabins")).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

